# Few-Shot과 Divide-and-Prompt를 활용한 Text-to-SQL

안녕하세요! 이 노트북에서는 자연어 질문을 SQL 쿼리로 변환하는 두 가지 강력한 방법을 학습하겠습니다.

## 이 노트북을 통해 배울 수 있는 것
- **Few-Shot Learning**: 몇 개의 예시만으로 AI가 패턴을 학습하는 방법
- **Divide-and-Prompt**: 복잡한 문제를 단계별로 나누어 해결하는 전략
- **실제 데이터베이스 연결**: MySQL과 AI를 연결하여 실용적인 시스템 구축
- **자기 검증**: AI가 생성한 쿼리를 스스로 검토하고 수정하는 방법

## 배경 지식
**Text-to-SQL**은 "어떤 직원이 가장 높은 급여를 받나요?"와 같은 자연어 질문을 `SELECT * FROM employees ORDER BY salary DESC LIMIT 1`과 같은 SQL 쿼리로 변환하는 기술입니다.

이 기술이 중요한 이유는 SQL을 모르는 사람도 데이터베이스에서 필요한 정보를 쉽게 찾을 수 있게 해주기 때문입니다.

---

## 실습 환경 권장사항
이 노트북을 원활하게 실행하기 위한 권장 환경입니다.

**Amazon SageMaker를 사용하는 경우:**
- SageMaker 이미지: `sagemaker-distribution-cpu`
- 커널: `Python 3`
- 인스턴스 타입: `ml.m5.large`

**참고:** 이 설정은 MySQL 데이터베이스 연결과 AI 모델 실행에 충분한 성능을 제공합니다.

---

## 목차

1. [종속성 설치](#1단계-종속성-설치)
1. [Bedrock과 데이터베이스 설정](#2단계-bedrock-클라이언트와-데이터베이스-연결-설정)
1. [데이터베이스 구축](#3단계-데이터베이스-구축)
1. [헬퍼 함수 생성](#4단계-헬퍼-함수-생성)
1. [Few-Shot 프롬프트 실행](#5단계-few-shot-프롬프트-실행)
1. [Divide & Prompt 프롬프트 생성](#6단계-divide--prompt-프롬프트-생성)
1. [Divide & Prompt 프롬프트 실행](#7단계-divide--prompt-프롬프트-실행)

---

## 학습 목표
이 노트북을 완료하면 다음을 할 수 있게 됩니다:

1. **Few-Shot Text-to-SQL 이해 및 실습**
   - 몇 개의 예시만으로 AI가 SQL을 생성하는 방법 학습
   - 프롬프트 엔지니어링 기법 적용

2. **Divide-and-Prompt 전략 구현**
   - 복잡한 문제를 단계별로 분해하는 방법
   - 자기 검증을 통한 정확도 향상

3. **실제 데이터베이스와 AI 연동**
   - MySQL 데이터베이스 설정 및 연결
   - 생성된 SQL 쿼리의 실제 실행 및 결과 확인

4. **프롬프트 최적화 기법**
   - 효과적인 프롬프트 템플릿 설계
   - AI 응답의 품질 향상 방법

## 실무 활용 가능성
- **비즈니스 인텔리전스**: 비개발자도 쉽게 데이터 분석
- **고객 지원**: 자동화된 데이터 조회 서비스 구축
- **보고서 자동화**: 자연어 요청으로 즉시 보고서 생성

---

## Text-to-SQL 두 가지 접근법 소개

이 노트북에서는 자연어를 SQL로 변환하는 두 가지 혁신적인 방법을 학습합니다:

### 1. Few-Shot Text-to-SQL
**개념:** 몇 개의 예시만을 사용하여 AI가 패턴을 학습하고 새로운 질문에 대한 SQL을 생성하는 방법

**작동 원리:**
- 자연어 질문과 해당 SQL 쿼리가 쌍을 이룬 예시 제공
- AI가 이 예시들로부터 패턴을 학습
- 새로운 질문에 대해 학습한 패턴을 적용하여 SQL 생성

**장점:**
- 간단하고 직관적인 접근법
- 적은 데이터로도 효과적인 학습 가능
- 빠른 구현과 테스트 가능

### 2. Divide-and-Prompt
**개념:** 복잡한 문제를 여러 단계로 나누어 각각을 순차적으로 해결하는 전략

**작동 원리:**
1. **1단계**: 초기 SQL 쿼리 생성
2. **2단계**: 생성된 쿼리의 정확성 검증
3. **3단계**: 필요시 오류 수정 및 최적화
4. **4단계**: 최종 검증

**장점:**
- 높은 정확도와 신뢰성
- 자기 검증을 통한 오류 감소
- 복잡한 쿼리에서 특히 효과적

### 연구 배경
- **Few-Shot 방법**: ["대형 언어 모델의 Few-shot Text-to-SQL 기능 향상"](https://arxiv.org/abs/2305.12586) 논문 기반
- **Divide-and-Prompt**: ["Divide-and-Prompt"](https://arxiv.org/abs/2304.11556) 연구 기반

### 사용할 도구들
- **LangChain**: 프롬프트 템플릿 관리 및 체인 구성
- **Amazon Bedrock SDK (Boto3)**: AWS AI 서비스 접근
- **MySQL**: 실제 데이터베이스 연결 및 쿼리 실행

---

### 1단계: 필수 라이브러리 설치

실습을 시작하기 전에 필요한 모든 라이브러리를 설치합니다.

#### 설치할 라이브러리 설명
- **boto3**: AWS 서비스 (Bedrock) 접근을 위한 SDK
- **botocore**: boto3의 핵심 라이브러리
- **langchain**: 프롬프트 템플릿과 AI 체인 관리 도구
- **sqlalchemy**: 데이터베이스 연결 및 ORM 지원
- **mysql-connector-python**: MySQL 데이터베이스 연결 전용 드라이버

#### 의존성 충돌 안내
설치 중 다음과 같은 의존성 충돌 경고가 나타날 수 있지만, 이는 다른 라이브러리와의 버전 충돌로 이번 실습에는 영향을 주지 않으므로 **무시하셔도 됩니다**:

```
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed...
```

이러한 경고는 시스템에 이미 설치된 다른 라이브러리들과의 버전 호환성 문제이며, 우리의 실습 기능에는 영향을 주지 않습니다.

In [ ]:
!python -m ensurepip --upgrade
%pip install -qU boto3
%pip install -qU botocore
%pip install -qU langchain
%pip install -qU sqlalchemy
%pip install -qU mysql-connector-python

#### 라이브러리 가져오기

설치가 완료되면 필요한 모든 모듈을 가져옵니다.

**가져올 모듈들의 역할:**
- `json, time, os, sys, random`: 파이썬 기본 유틸리티
- `functools.partial`: 함수 부분 적용 (설정값 미리 지정)
- `langchain.PromptTemplate`: 프롬프트 템플릿 관리
- `boto3`: AWS Bedrock AI 서비스 접근
- `sqlalchemy`: 데이터베이스 연결 추상화
- `mysql.connector`: MySQL 직접 연결
- `utilities`: 워크샵용 헬퍼 함수들

이 모듈들을 통해 AI 모델과 데이터베이스를 연결하는 완전한 시스템을 구축할 수 있습니다.

In [ ]:
import json
import time
import os
import sys
import random
import importlib
from functools import partial

from langchain import PromptTemplate
import boto3
import sqlalchemy
from sqlalchemy import create_engine
import mysql.connector

sys.path.append('../')
import utilities as u

#### AI 모델 설정

사용할 AI 모델과 관련 파라미터를 설정합니다.

**설정 값 설명:**
- **model_id**: 사용할 AI 모델 식별자
  - `anthropic.claude-v2`: 복잡한 추론과 SQL 생성에 뛰어난 Claude V2 모델
  - `amazon.titan-tg1-large`: (주석 처리됨) Amazon의 Titan 모델 대안

- **temperature**: AI 응답의 창의성/일관성 조절 (0.0~1.0)
  - `0.2`: 낮은 값으로 일관되고 정확한 SQL 생성에 적합
  - 값이 낮을수록 더 결정적이고 예측 가능한 응답

- **top_k**: 다음 토큰 선택 시 고려할 후보 수
  - `200`: 적당한 다양성을 유지하면서도 품질 보장

**왜 이 설정인가?**
SQL 생성은 정확성이 중요하므로 낮은 temperature로 일관된 결과를 얻고, Claude V2는 복잡한 논리 추론에 특화되어 있어 Text-to-SQL 작업에 적합합니다.

In [ ]:
model_id: str = "anthropic.claude-v2"
# model_id: str = "amazon.titan-tg1-large"
temperature: float = 0.2
top_k: int = 200

#### Bedrock 호출 함수 준비

`partial` 함수를 사용하여 Bedrock API 호출을 위한 편리한 함수를 미리 설정합니다.

**partial 함수의 장점:**
- 자주 사용되는 설정값들을 미리 지정
- 매번 동일한 파라미터를 반복 입력할 필요 없음
- 코드의 간결성과 일관성 향상

**설정된 파라미터:**
- `system_prompts=[]`: 시스템 프롬프트 없음 (필요시 나중에 추가)
- `model_id`, `temperature`, `top_k`: 위에서 설정한 값들 자동 적용

**사용법 예시:**
이제 `run_bedrock(prompt="질문")`만으로 간단하게 AI를 호출할 수 있습니다.

이는 반복적인 AI 호출 코드를 크게 단순화해줍니다.

In [ ]:
run_bedrock = partial(u.run_bedrock_simple_prompt,
                      system_prompts=[],
                      model_id=model_id,
                      temperature=temperature,
                      top_k=top_k)

### 2단계: Bedrock 클라이언트와 데이터베이스 연결 설정

이제 AWS Bedrock AI 서비스와 MySQL 데이터베이스에 연결하겠습니다.

#### 두 가지 연결이 필요한 이유
1. **Bedrock 연결**: 자연어를 SQL로 변환하는 AI 서비스
2. **MySQL 연결**: 생성된 SQL을 실제로 실행하여 결과 확인

#### 연결 과정
1. Bedrock 클라이언트 초기화
2. CloudFormation에서 데이터베이스 연결 정보 추출
3. MySQL 데이터베이스 연결 수립
4. 연결 상태 확인

이 두 연결을 통해 완전한 Text-to-SQL 시스템이 구축됩니다.

In [ ]:
# AWS Bedrock 클라이언트 초기화
bedrock_client = boto3.client(service_name='bedrock-runtime')

In [ ]:
# CloudFormation에서 데이터베이스 연결 정보 추출
# 워크샵 설정에서 자동으로 생성된 MySQL 서버 정보를 가져옵니다
DB_HOST, DB_USER, DB_PASSWORD = \
    u.extract_CF_outputs("RDSInstanceEndpoint", "DbUser", "DbPassword")

print("데이터베이스 연결 정보:")
print(f"호스트: {DB_HOST}")
print(f"사용자: {DB_USER}")
print("비밀번호: [보안상 숨김]")

DB_HOST, DB_USER, DB_PASSWORD

In [ ]:
# MySQL 데이터베이스 연결 수립
mydb = mysql.connector.connect(
    host=DB_HOST,      # RDS 인스턴스 엔드포인트
    user=DB_USER,      # 데이터베이스 사용자명
    password=DB_PASSWORD  # 데이터베이스 비밀번호
)

print("MySQL 연결 성공!")
print(f"연결 객체: {mydb}")
mydb

#### 데이터베이스 연결 확인

연결이 성공적으로 이루어졌는지 확인하기 위해 현재 MySQL 서버에 있는 모든 데이터베이스를 조회해보겠습니다.

**확인 목적:**
- 데이터베이스 연결이 정상적으로 작동하는지 테스트
- 기존에 있는 데이터베이스들 파악
- 다음 단계에서 생성할 데이터베이스와 충돌 방지

In [ ]:
# 데이터베이스 목록 조회
mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")

print("현재 MySQL 서버의 데이터베이스 목록:")
for x in mycursor:
    print(f"- {x[0]}")  # 데이터베이스 이름 출력

### 3단계: 실습용 데이터베이스 구축

이제 Text-to-SQL 실습을 위한 전용 데이터베이스와 테이블을 생성하겠습니다.

#### 데이터베이스 구축 과정
1. **기존 데이터 정리**: 실습용 테이블과 데이터베이스가 있다면 삭제
2. **새 데이터베이스 생성**: `LLM_DEMO`라는 이름으로 생성
3. **테이블 생성**: 직원 정보를 담을 `TEST_EMPLOYEE_LLM` 테이블 생성
4. **샘플 데이터 삽입**: 실습용 가상 직원 데이터 20개 삽입

#### 왜 가상 데이터를 사용하나요?
- **학습 목적**: 복잡하지 않은 구조로 개념 이해에 집중
- **안전성**: 실제 민감한 데이터 노출 위험 없음
- **재현 가능성**: 모든 사람이 동일한 환경에서 실습 가능
- **다양성**: 여러 유형의 SQL 쿼리 연습 가능

실습용 데이터이므로 직원 이름들이 재미있게 만들어져 있습니다!

In [ ]:
# 1단계: 기존 테이블 삭제 (있다면)
print("기존 테이블 정리 중...")
mycursor.execute("DROP TABLE IF EXISTS LLM_DEMO.TEST_EMPLOYEE_LLM")
print("✓ 기존 테이블 삭제 완료")

In [ ]:
# 2단계: 기존 데이터베이스 삭제 (있다면)
print("기존 데이터베이스 정리 중...")
mycursor.execute("DROP DATABASE IF EXISTS LLM_DEMO")
print("✓ 기존 데이터베이스 삭제 완료")

# 3단계: 새로운 `LLM_DEMO` 데이터베이스 생성

Text-to-SQL 실습을 위한 전용 데이터베이스를 만듭니다.

In [ ]:
print("새 데이터베이스 생성 중...")
mycursor.execute("CREATE DATABASE LLM_DEMO")
print("✓ LLM_DEMO 데이터베이스 생성 완료!")

# 4단계: 직원 정보 테이블 생성

가상의 직원 정보를 담을 `TEST_EMPLOYEE_LLM` 테이블을 생성합니다.

#### 테이블 구조 설명
각 컬럼의 의미를 이해하면 나중에 자연어 질문을 만들 때 도움이 됩니다:

- **EMPID**: 직원 고유 번호
- **NAME**: 직원 이름  
- **SALARY**: 기본 급여
- **BONUS**: 보너스
- **CITY**: 근무 도시
- **JOINING_DATE**: 입사일
- **ACTIVE_EMPLOYEE**: 재직 여부 (1=재직, 0=퇴사)
- **DEPARTMENT**: 소속 부서
- **TITLE**: 직급/직책

In [ ]:
print("직원 테이블 생성 중...")
mycursor.execute("""
CREATE TABLE LLM_DEMO.TEST_EMPLOYEE_LLM -- 테이블 이름
(EMPID INT(10), -- 직원 ID
NAME VARCHAR(20), -- 직원 이름
SALARY INT(10), -- 기본 급여
BONUS INT(10), -- 보너스
CITY VARCHAR(20), -- 근무 도시
JOINING_DATE TIMESTAMP, -- 입사일
ACTIVE_EMPLOYEE INT(2), -- 재직 여부 (1=재직, 0=퇴사)
DEPARTMENT VARCHAR(20), -- 소속 부서
TITLE VARCHAR(20) -- 직급/직책
)
""")
print("✓ TEST_EMPLOYEE_LLM 테이블 생성 완료!")

# 5단계: 실습용 샘플 데이터 삽입

다양한 SQL 쿼리를 연습할 수 있도록 20명의 가상 직원 데이터를 삽입합니다.

#### 데이터 특징
- **다양한 부서**: Engineering, Sales, Marketing, IT, Finance
- **다양한 직급**: Manager, Executive, Associate, Lead, Director, Analyst
- **다양한 도시**: 미국 주요 도시들
- **다양한 급여 범위**: $45,000 ~ $90,000
- **다양한 입사 연도**: 2013년 ~ 2022년

이렇게 다양한 데이터로 집계, 정렬, 필터링 등 다양한 유형의 SQL 쿼리를 연습할 수 있습니다.

In [ ]:
print("샘플 데이터 삽입 중...")

# 20명의 가상 직원 데이터 삽입
employees_data = [
    (1, 'Xyon McFluff', 50000, 10000, 'New York', '2020-01-01 10:00:00', 1, 'Engineering', 'Manager'),
    (2, 'Twinkle Luna', 60000, 5000, 'Chicago', '2018-05-15 11:30:00', 1, 'Sales', 'Executive'),
    (3, 'Zorfendorf', 45000, 2000, 'Miami', '2021-09-01 09:15:00', 1, 'Marketing', 'Associate'),
    (4, 'Gloobinorg', 72000, 8000, 'Seattle', '2017-04-05 14:20:00', 1, 'IT', 'Manager'),
    (5, 'Bonkliwop', 65000, 6000, 'Denver', '2020-11-24 08:45:00', 1, 'Sales', 'Associate'),
    (6, 'Ploopdewoop', 55000, 4000, 'Philadelphia', '2019-03-11 10:25:00', 1, 'Marketing', 'Executive'),
    (7, 'Flooblelobber', 80000, 9000, 'San Francisco', '2016-08-20 12:35:00', 1, 'Engineering', 'Lead'),
    (8, 'Blippitybloop', 57000, 3000, 'Boston', '2018-12-01 15:00:00', 1, 'Finance', 'Analyst'),
    (9, 'Snorkeldink', 74000, 7000, 'Atlanta', '2015-10-07 16:15:00', 1, 'IT', 'Lead'),
    (10, 'Wuggawugga', 69000, 5000, 'Austin', '2017-06-19 13:45:00', 1, 'Engineering', 'Manager'),
    (11, 'Foofletoot', 62000, 4000, 'San Diego', '2019-02-24 17:30:00', 1, 'Sales', 'Associate'),
    (12, 'Bonkbonk', 82000, 8000, 'Silicon Valley', '2014-12-05 09:45:00', 1, 'Engineering', 'Director'),
    (13, 'Zippityzoom', 78000, 7500, 'New York', '2016-03-08 11:00:00', 1, 'IT', 'Manager'),
    (14, 'Splatchsplatch', 90000, 9500, 'Chicago', '2013-01-26 13:15:00', 1, 'Marketing', 'Director'),
    (15, 'Wuggles', 85000, 8000, 'Seattle', '2018-07-22 15:30:00', 1, 'Finance', 'Manager'),
    (16, 'Boingboing', 70000, 6000, 'Miami', '2020-04-11 16:45:00', 1, 'Sales', 'Lead'),
    (17, 'Zipzoom', 62000, 5000, 'Denver', '2017-09-18 18:00:00', 1, 'Engineering', 'Associate'),
    (18, 'Wooglewoogle', 58000, 3500, 'Philadelphia', '2019-12-24 08:20:00', 1, 'IT', 'Analyst'),
    (19, 'Flipflopglop', 75000, 7200, 'Boston', '2022-02-14 10:35:00', 1, 'Marketing', 'Lead'),
    (20, 'Blipblop', 68000, 6500, 'San Francisco', '2021-08-29 11:50:00', 1, 'Finance', 'Executive')
]

# 데이터 일괄 삽입
insert_query = """
INSERT INTO LLM_DEMO.TEST_EMPLOYEE_LLM 
(EMPID, NAME, SALARY, BONUS, CITY, JOINING_DATE, ACTIVE_EMPLOYEE, DEPARTMENT, TITLE)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

mycursor.executemany(insert_query, employees_data)
mydb.commit()

print(f"✓ {len(employees_data)}명의 직원 데이터 삽입 완료!")
print("데이터베이스 구축이 완료되었습니다.")

# 6단계: 데이터베이스 구축 확인

데이터가 정상적으로 삽입되었는지 확인하기 위해 테이블의 모든 레코드를 조회해보겠습니다.

In [ ]:
print("데이터베이스 연결 및 데이터 확인:")
print("=" * 50)

mycursor.execute("SELECT * FROM LLM_DEMO.TEST_EMPLOYEE_LLM")
myresult = mycursor.fetchall()

print(f"총 {len(myresult)}명의 직원 데이터가 확인되었습니다.")
print("\n[샘플 데이터 - 처음 5명]")
print("ID | 이름          | 급여   | 보너스 | 도시        | 부서      | 직급")
print("-" * 70)

for i, x in enumerate(myresult[:5]):  # 처음 5명만 출력
    print(f"{x[0]:2} | {x[1]:12} | ${x[2]:5} | ${x[3]:5} | {x[4]:10} | {x[7]:8} | {x[8]}")

print(f"\n... (총 {len(myresult)}명의 데이터)")
print("✓ 데이터베이스 구축 및 연결 확인 완료!")

## Few-Shot Text-to-SQL 실습 시작

데이터베이스와 테이블 구축이 완료되었으니, 이제 **Few-Shot Text-to-SQL** 접근법을 실습해보겠습니다.

### Few-Shot Learning이란?
**Few-Shot Learning**은 몇 개의 예시만으로 AI가 새로운 패턴을 학습하는 방법입니다.

**전통적인 방법 vs Few-Shot:**
- **전통적 방법**: 수천 개의 데이터로 모델 훈련 필요
- **Few-Shot 방법**: 2-5개의 예시만으로 패턴 학습 가능

### Text-to-SQL에서의 적용
1. **예시 제공**: "질문 → SQL" 쌍을 몇 개 보여줌
2. **패턴 학습**: AI가 자연어와 SQL 간의 매핑 규칙 학습  
3. **새 질문 해결**: 학습한 패턴으로 새로운 질문에 대한 SQL 생성

### 다음 단계
먼저 SQL 실행과 결과 처리를 위한 헬퍼 함수들을 만들어보겠습니다.

### 4단계: 헬퍼 함수 생성

AI가 생성한 SQL을 실행하고 결과를 처리하기 위한 유틸리티 함수들을 만들겠습니다.

#### 첫 번째 함수: `call_database`
이 함수는 다음 역할을 수행합니다:
1. **SQL 실행**: AI가 생성한 SQL 쿼리를 데이터베이스에서 실행
2. **결과 가져오기**: 쿼리 실행 결과를 모두 가져옴
3. **형식 변환**: 결과를 문자열로 변환하여 다음 단계에서 사용하기 쉽게 만듦
4. **화면 출력**: 사용자가 결과를 즉시 확인할 수 있도록 출력

**왜 이 함수가 필요한가?**
- AI가 생성한 SQL이 실제로 작동하는지 즉시 확인
- 결과 데이터를 다음 처리 단계로 전달
- 에러 발생 시 문제점 파악 가능

In [ ]:
def call_database(llm_generated_response: str) -> str:
    """
    AI가 생성한 SQL 쿼리를 데이터베이스에서 실행하고 결과를 반환합니다.

    매개변수:
      llm_generated_response: AI가 생성한 SQL 쿼리 문자열
    반환값:
      SQL 실행 결과를 포맷팅한 문자열
    """
    print(f"실행할 SQL: {llm_generated_response}")
    print("-" * 50)
    
    mycursor = mydb.cursor()
    mycursor.execute(llm_generated_response)
    myresult = mycursor.fetchall()
    
    output_string = ''
    print("쿼리 실행 결과:")
    for x in myresult:
        output_string = output_string + str(x) + "\n"
        print(x)
    
    print(f"\n총 {len(myresult)}개의 결과를 찾았습니다.")
    return output_string

#### 두 번째 함수: `prepare_final_gen_text`

이 함수는 SQL 실행 결과를 사람이 이해하기 쉬운 자연어로 변환합니다.

**작동 과정:**
1. **템플릿 준비**: 미리 정의된 프롬프트 템플릿 사용
2. **데이터 결합**: 원래 질문과 SQL 실행 결과를 결합
3. **AI 호출**: 결합된 정보를 AI에게 전달
4. **자연어 생성**: AI가 기술적인 결과를 친근한 언어로 변환

**예시:**
- **SQL 결과**: `[(5, 'Engineering'), (3, 'Sales')]`
- **자연어 변환**: "Engineering 부서에 5명, Sales 부서에 3명의 직원이 있습니다."

**왜 이 단계가 중요한가?**
- 비개발자도 결과를 쉽게 이해
- 테이블이나 스키마 이름 등 기술적 용어 제거
- 더 자연스럽고 친근한 답변 제공

In [ ]:
def prepare_final_gen_text(prompt_final: str, output_string: str) -> str:
    """
    SQL 실행 결과를 사람이 이해하기 쉬운 자연어로 변환합니다.

    매개변수:
      prompt_final: 자연어 생성을 위한 프롬프트 템플릿
      output_string: SQL 실행 결과 문자열
    반환값:
      사용자 친화적인 자연어 답변
    """
    print("🤖 결과를 자연어로 변환 중...")
    
    # 프롬프트 템플릿에 데이터 삽입
    prompt_template_for_query_response = PromptTemplate.from_template(prompt_final)
    prompt_data_for_query_response = prompt_template_for_query_response.format(
        question=question_asked, 
        answer=output_string
    )
    
    # AI에게 자연어 생성 요청
    final_response_text = run_bedrock(prompt=prompt_data_for_query_response)
    
    print("최종 답변:")
    print(final_response_text)
    
    return final_response_text

### 5단계: Few-Shot 프롬프트 템플릿 생성

이제 Few-Shot Text-to-SQL의 핵심인 프롬프트 템플릿을 설계합니다.

#### 프롬프트 설계 원칙
효과적인 Few-Shot 프롬프트는 다음 요소들을 포함해야 합니다:

1. **명확한 역할 정의**: AI에게 "MySQL 쿼리 전문가" 역할 부여
2. **상세한 스키마 정보**: 테이블 구조와 컬럼 설명 제공
3. **구체적인 예시**: 샘플 데이터를 통한 패턴 학습 지원
4. **출력 형식 지정**: `<sql></sql>` 태그로 결과 구조화
5. **제약 조건 명시**: 사용 가능한 테이블 제한

#### 첫 번째 템플릿: 결과를 자연어로 변환
SQL 실행 결과를 사용자 친화적인 언어로 변환하는 템플릿부터 만들어보겠습니다.

In [ ]:
# 자연어 변환을 위한 프롬프트 템플릿
prompt_final = """
Human: 다음 질문에 대한 답변을 바탕으로 사용자 친화적인 답변을 제공해주세요.

질문: {question}

데이터베이스 조회 결과: {answer}

요구사항:
- 간단하고 명확한 한국어로 답변
- 테이블명이나 스키마명은 포함하지 말 것
- 숫자나 데이터는 이해하기 쉽게 설명

#### 두 번째 템플릿: Few-Shot SQL 생성

이제 Few-Shot Learning의 핵심인 SQL 생성 프롬프트를 만들어보겠습니다.

**이 프롬프트의 특징:**
- **Single-Shot 예시**: 하나의 구체적인 예시로 패턴 학습
- **상세한 스키마**: 각 컬럼의 의미와 용도 명확히 설명
- **샘플 데이터**: 실제 데이터 예시로 이해도 증진
- **구조화된 출력**: `<sql></sql>` 태그로 결과 파싱 용이성 확보

이 템플릿이 AI에게 어떤 힌트를 제공하는지 주의 깊게 살펴보세요.

In [ ]:
# Few-Shot Text-to-SQL을 위한 메인 프롬프트 템플릿
prompt = """
당신은 MySQL 쿼리 전문가이며, 유효한 SQL 쿼리만을 출력합니다.

다음 테이블만 사용하세요:

테이블 스키마:
<table_schema>
CREATE TABLE LLM_DEMO.TEST_EMPLOYEE_LLM -- 직원 테이블
(EMPID INT(10), -- 직원 고유 ID
NAME VARCHAR(20), -- 직원 이름
SALARY INT(10), -- 기본 급여 (연봉)
BONUS INT(10), -- 보너스 금액
CITY VARCHAR(20), -- 근무 도시
JOINING_DATE TIMESTAMP, -- 입사일
ACTIVE_EMPLOYEE INT(2), -- 재직 상태 (1=재직, 0=퇴사)
DEPARTMENT VARCHAR(20), -- 소속 부서
TITLE VARCHAR(20) -- 직급/직책
)
</table_schema>

스키마 이름: LLM_DEMO

참고용 샘플 레코드:
INSERT INTO LLM_DEMO.TEST_EMPLOYEE_LLM (EMPID, NAME, SALARY, BONUS, CITY, JOINING_DATE, ACTIVE_EMPLOYEE, DEPARTMENT, TITLE) 
VALUES (1, 'Stuart', 25000, 5000, 'Seattle', '2023-01-21 00:00:01', 1, 'Applications', 'Sr. Developer');

지시사항:
- 위 테이블 구조를 정확히 사용하여 유효한 MySQL 쿼리를 작성하세요
- 결과는 반드시 <sql></sql> 태그 안에 넣어주세요
- 오직 MySQL 쿼리만 반환하세요

질문: {question}
"""

### 6단계: Few-Shot 프롬프트 실행

이제 준비된 Few-Shot 프롬프트를 실제로 실행해보겠습니다!

#### 실행 과정
1. **자연어 질문 입력**: 일상적인 언어로 질문 작성
2. **프롬프트 생성**: 템플릿에 질문 삽입
3. **AI SQL 생성**: Claude가 SQL 쿼리 생성
4. **SQL 실행**: 생성된 쿼리를 데이터베이스에서 실행
5. **결과 변환**: SQL 결과를 자연어로 변환

다음 셀들에서 다양한 질문들과 AI가 생성한 SQL을 확인할 수 있습니다.
생성된 SQL은 `<sql></sql>` 태그 안에 포함되어 출력됩니다.

#### 첫 번째 질문: 부서별 활성 직원 수

**질문**: "각 부서에서 활성 상태인 직원의 총 수는 얼마인가요?"

이는 GROUP BY와 COUNT 함수를 사용하는 집계 쿼리의 전형적인 예시입니다.

In [ ]:
print("=" * 60)
print("첫 번째 Few-Shot 실습")
print("=" * 60)

question_asked = "What is the total count of employees who are active in each department?"

print(f"질문: {question_asked}")
print("\n1️AI가 SQL 생성 중...")

# 프롬프트 템플릿에 질문 삽입
prompt_template_for_query_generate = PromptTemplate.from_template(prompt)
prompt_data_for_query_generate = prompt_template_for_query_generate.format(question=question_asked)

# AI에게 SQL 생성 요청
llm_generated_response = run_bedrock(prompt=prompt_data_for_query_generate)
print("AI 응답:")
print(llm_generated_response)

# SQL 쿼리 추출
llm_generated_sql = u.extract_tag(llm_generated_response, "sql")[0]
print(f"\n추출된 SQL: {llm_generated_sql}")

print("\n2️SQL 실행 중...")
# SQL 실행
output_string = call_database(llm_generated_sql)

print("\n3️최종 답변 생성 중...")
# 자연어 답변 생성
final_answer = prepare_final_gen_text(prompt_final, output_string)

print("\n" + "=" * 60)
print("첫 번째 실습 완료!")
print("=" * 60)

#### 두 번째 질문: 부서별 평균 급여

**질문**: "각 부서 직원들의 평균 급여는 얼마인가요?"

이번에는 AVG 함수를 사용하는 집계 쿼리입니다. Few-Shot Learning이 다른 유형의 집계 함수도 잘 처리하는지 확인해보겠습니다.

In [ ]:


print("\n" + "=" * 60)
print("두 번째 Few-Shot 실습")
print("=" * 60)

question_asked = "What is the average salary of employees in each department?"

print(f"질문: {question_asked}")
print("\n1️AI가 SQL 생성 중...")

prompt_template_for_query_generate = PromptTemplate.from_template(prompt)
prompt_data_for_query_generate = prompt_template_for_query_generate.format(question=question_asked)

llm_generated_response = run_bedrock(prompt=prompt_data_for_query_generate)
print("AI 응답:")
print(llm_generated_response)

# SQL 추출 및 실행
llm_generated_sql = u.extract_tag(llm_generated_response, "sql")[0]
print(f"\n추출된 SQL: {llm_generated_sql}")

print("\n2️SQL 실행 중...")
output_string = call_database(llm_generated_sql)

print("\n3️최종 답변 생성 중...")
final_answer = prepare_final_gen_text(prompt_final, output_string)

print("\n" + "=" * 60)
print("두 번째 실습 완료!")
print("=" * 60)

# Divide-and-Prompt: 단계별 사고를 통한 Text-to-SQL

Few-Shot 방법을 실습해보았으니, 이제 두 번째 접근법인 **Divide-and-Prompt**를 배워보겠습니다.

## Divide-and-Prompt란?
**Divide-and-Prompt**는 복잡한 문제를 여러 단계로 나누어 각각을 체계적으로 해결하는 방법입니다.

### 기본 원리
1. **초기 시도**: 첫 번째 프롬프트로 SQL 생성
2. **검증 단계**: 두 번째 프롬프트로 결과 검토
3. **수정 및 개선**: 필요시 오류 수정 및 최적화
4. **최종 확인**: 수정된 결과의 정확성 검증

### Few-Shot vs Divide-and-Prompt
| 구분 | Few-Shot | Divide-and-Prompt |
|------|----------|-------------------|
| **접근 방식** | 단일 단계 | 다중 단계 |
| **정확도** | 보통 | 높음 |
| **처리 시간** | 빠름 | 느림 |
| **복잡한 쿼리** | 제한적 | 우수 |
| **자기 검증** | 없음 | 있음 |

### 7단계: Divide-and-Prompt 검증 템플릿 생성

Divide-and-Prompt의 핵심은 **자기 검증** 기능입니다. 첫 번째 단계에서 생성된 SQL을 두 번째 단계에서 검토하고 수정하는 프롬프트를 만들어보겠습니다.

#### 검증 프롬프트의 특징
1. **이중 검증 시스템**: "올바르면 그대로, 틀리면 수정"
2. **구체적인 스키마 재확인**: 테이블과 컬럼 정보 다시 제공
3. **명확한 판단 기준**: 정확성 판단을 위한 체크리스트
4. **수정 지침**: 오류 발견 시 정확한 수정 방향 제시

이 템플릿은 AI가 SQL 전문가로서 다른 AI가 생성한 쿼리를 검토하는 역할을 수행하도록 설계되었습니다.

In [ ]:
# Divide-and-Prompt용 검증 및 수정 프롬프트 템플릿
prompt_check_modify_sql = """
당신은 MySQL 쿼리 검증 전문가입니다. 주어진 SQL의 정확성을 검토하고 필요시 수정해주세요.

테이블 스키마:
<table_schema>
CREATE TABLE LLM_DEMO.TEST_EMPLOYEE_LLM -- 직원 테이블
(EMPID INT(10), -- 직원 고유 ID 컬럼
NAME VARCHAR(20), -- 직원 이름 컬럼
SALARY INT(10), -- 기본 급여 컬럼 (연봉)
BONUS INT(10), -- 보너스 컬럼
CITY VARCHAR(20), -- 근무 도시 컬럼
JOINING_DATE TIMESTAMP, -- 입사일 컬럼
ACTIVE_EMPLOYEE INT(2), -- 재직 상태 컬럼 (1=재직, 0=퇴사)
DEPARTMENT VARCHAR(20), -- 소속 부서 컬럼
TITLE VARCHAR(20) -- 직급/직책 컬럼
)
</table_schema>

사용 가능한 테이블(컬럼):
LLM_DEMO.TEST_EMPLOYEE_LLM(EMPID,NAME,SALARY,BONUS,CITY,JOINING_DATE,ACTIVE_EMPLOYEE,DEPARTMENT,TITLE)

검토 대상:
원래 질문: {question_asked}
생성된 SQL: {llm_generated_response}

검증 지침:
1. 테이블명과 컬럼명이 스키마와 일치하는가?
2. SQL 문법이 올바른가?
3. 질문의 의도를 정확히 반영하는가?
4. 집계 함수나 조건절이 적절한가?

판단 결과:
- SQL이 올바르다면: "이 SQL은 정확합니다" 라고만 답변
- SQL에 오류가 있다면: 수정된 SQL을 <sql></sql> 태그 안에 제공
"""

### 8단계: Divide-and-Prompt 실행

이제 Divide-and-Prompt의 2단계 프로세스를 실제로 실행해보겠습니다.

#### 실행 순서
1. **1단계**: Few-Shot 템플릿으로 초기 SQL 생성
2. **2단계**: 검증 템플릿으로 SQL 검토 및 수정
3. **결과 비교**: 1단계와 2단계 결과의 차이점 분석

#### 테스트 질문
좀 더 복잡한 질문으로 Divide-and-Prompt의 효과를 확인해보겠습니다.
**질문**: "Lead 직급을 가진 직원이 가장 많은 도시는 어디인가요?"

이 질문은 특정 조건 필터링과 집계를 동시에 요구하므로 실수가 발생하기 쉬운 유형입니다.

In [ ]:
#### 1단계: 초기 SQL 생성

print("\n" + "=" * 70)
print("Divide-and-Prompt 실습 시작")
print("=" * 70)

question_asked = "Which city has the most employees with a title of Lead?"

print(f"질문: {question_asked}")
print(f"한국어: Lead 직급 직원이 가장 많은 도시는?")

print("\n1단계: Few-Shot으로 초기 SQL 생성")
print("-" * 50)

prompt_template_for_query_generate = PromptTemplate.from_template(prompt)
prompt_data_for_query_generate = prompt_template_for_query_generate.format(question=question_asked)

llm_generated_response = run_bedrock(prompt=prompt_data_for_query_generate)
print("1단계 AI 응답:")
print(llm_generated_response)

# SQL 추출
llm_generated_sql = u.extract_tag(llm_generated_response, "sql")[0]
print(f"\n1단계에서 생성된 SQL:")
print(llm_generated_sql)

print("\n1단계 완료 - 초기 SQL 생성됨")
print("다음: 2단계에서 이 SQL을 검증하고 수정합니다.")

#### 2단계: SQL 검증 및 수정

이제 Divide-and-Prompt의 핵심인 **검증 단계**를 실행합니다. 
첫 번째 단계에서 생성된 SQL을 두 번째 AI가 검토하여 오류를 찾고 수정합니다.

**검증 과정:**
- 테이블명과 컬럼명 정확성 확인
- SQL 문법 검증
- 비즈니스 로직 정확성 평가
- 필요시 개선된 SQL 제공

In [ ]:
print("\n🔍 2단계: SQL 검증 및 수정")
print("-" * 50)

# 검증 프롬프트 생성
prompt_template_for_query_generate = PromptTemplate.from_template(prompt_check_modify_sql)
prompt_data_for_query_generate = prompt_template_for_query_generate.format(
    question_asked=question_asked,
    llm_generated_response=llm_generated_sql
)

# AI에게 검증 요청
llm_generated_response = run_bedrock(prompt=prompt_data_for_query_generate)
print("2단계 AI 검증 결과:")
print(llm_generated_response)

# 결과 분석
if "정확합니다" in llm_generated_response or "correct" in llm_generated_response.lower():
    print("\n검증 결과: 1단계 SQL이 정확합니다!")
    final_sql = llm_generated_sql
else:
    print("\n🔧 검증 결과: SQL 수정이 필요합니다.")
    try:
        final_sql = u.extract_tag(llm_generated_response, "sql")[0]
        print(f"수정된 SQL:")
        print(final_sql)
    except:
        print("⚠️ 수정된 SQL을 추출할 수 없습니다. 원본 SQL을 사용합니다.")
        final_sql = llm_generated_sql

print(f"\n최종 사용할 SQL:")
print(final_sql)

print("\n2단계 완료 - SQL 검증 및 수정 완료")
print("=" * 70)

#### 추가 실습: 다른 질문으로 전체 프로세스 실행

Divide-and-Prompt의 효과를 더 확실히 확인하기 위해 다른 질문으로도 실습해보겠습니다.

**새로운 질문**: "각 부서별 입사일 범위는 어떻게 되나요?"
이 질문은 날짜 함수와 집계를 함께 사용하는 복잡한 쿼리입니다.

In [ ]:
print("\n" + "🔄" * 30)
print("추가 실습: 복잡한 날짜 쿼리")
print("🔄" * 30)

question_asked = "What is the range of joining dates for employees in each department?"

print(f"질문: {question_asked}")
print(f"한국어: 각 부서 직원들의 입사일 범위는?")

print("\n1단계: 초기 SQL 생성")
prompt_template_for_query_generate = PromptTemplate.from_template(prompt)
prompt_data_for_query_generate = prompt_template_for_query_generate.format(question=question_asked)

llm_generated_response = run_bedrock(prompt=prompt_data_for_query_generate)
print("1단계 결과:")
print(llm_generated_response)

# SQL 추출
llm_generated_sql = u.extract_tag(llm_generated_response, "sql")[0]
print(f"\n생성된 SQL: {llm_generated_sql}")

print("\n2단계: SQL 검증")
prompt_template_for_query_generate = PromptTemplate.from_template(prompt_check_modify_sql)
prompt_data_for_query_generate = prompt_template_for_query_generate.format(
    question_asked=question_asked,
    llm_generated_response=llm_generated_sql
)

llm_generated_response = run_bedrock(prompt=prompt_data_for_query_generate)
print("2단계 검증 결과:")
print(llm_generated_response)

print("\n" + "=" * 60)
print("추가 실습 완료!")
print("=" * 60)

#### 오류 시나리오 테스트: 의도적인 잘못된 SQL

Divide-and-Prompt의 오류 수정 능력을 확인하기 위해 의도적으로 잘못된 SQL을 제공하고 AI가 이를 올바르게 수정하는지 테스트해보겠습니다.

**테스트 시나리오:**
- 잘못된 테이블명 사용 (`TEST_EMPLOYEE_LLM_TEST` → 존재하지 않음)
- 잘못된 컬럼명 사용 (`AGE` → 존재하지 않음)
- 이런 오류들을 AI가 감지하고 수정할 수 있는지 확인

In [ ]:
print("\n" + "⚠️" * 30)
print("오류 수정 능력 테스트")
print("⚠️" * 30)

# 의도적으로 잘못된 SQL 생성
question_asked = "What is the range of joining dates for employees in each department?"
print(f"질문: {question_asked}")

# 여러 오류가 있는 SQL (테이블명 오류, 컬럼명 오류)
llm_generated_sql = """
SELECT DEPARTMENT, AGE, MIN(JOINING_DATE) AS Earliest_Join_Date, MAX(JOINING_DATE) AS Latest_Join_Date
FROM LLM_DEMO.TEST_EMPLOYEE_LLM_TEST
GROUP BY DEPARTMENT
"""

print("\n의도적으로 만든 잘못된 SQL:")
print(llm_generated_sql)

print("\n오류 내용:")
print("1. 존재하지 않는 테이블명: TEST_EMPLOYEE_LLM_TEST")
print("2. 존재하지 않는 컬럼명: AGE")
print("3. 올바른 테이블명: TEST_EMPLOYEE_LLM")

print("\nAI가 이 오류들을 찾아서 수정할 수 있을까요?")
print("-" * 50)

# Divide-and-Prompt 검증 실행
prompt_template_for_query_generate = PromptTemplate.from_template(prompt_check_modify_sql)
prompt_data_for_query_generate = prompt_template_for_query_generate.format(
    question_asked=question_asked,
    llm_generated_response=llm_generated_sql
)

llm_generated_response = run_bedrock(prompt=prompt_data_for_query_generate)
print("AI 검증 및 수정 결과:")
print(llm_generated_response)

# 수정된 SQL 추출 시도
try:
    corrected_sql = u.extract_tag(llm_generated_response, "sql")[0]
    print(f"\n수정된 SQL:")
    print(corrected_sql)
    
    print(f"\n수정 내용 분석:")
    if "TEST_EMPLOYEE_LLM_TEST" not in corrected_sql:
        print("테이블명 오류 수정됨")
    if "AGE" not in corrected_sql:
        print("잘못된 컬럼명 제거됨")
        
except Exception as e:
    print(f"수정된 SQL 추출 실패: {e}")

print("\n" + "=" * 60)
print("오류 수정 테스트 완료!")
print("=" * 60)

## 실습 완료

축하합니다! Few-Shot과 Divide-and-Prompt 두 가지 Text-to-SQL 방법을 모두 실습해보았습니다.

### 학습한 내용 요약

#### Few-Shot Text-to-SQL
- **원리**: 몇 개의 예시로 패턴 학습
- **장점**: 빠르고 간단한 구현
- **적용**: 단순하고 일반적인 쿼리에 효과적

#### Divide-and-Prompt
- **원리**: 2단계 검증을 통한 정확도 향상
- **장점**: 높은 정확도와 자기 검증 기능
- **적용**: 복잡하고 중요한 쿼리에 효과적

### 실무 적용 가이드

**Few-Shot을 사용하는 경우:**
- 프로토타입 개발
- 간단한 쿼리 생성
- 빠른 응답이 필요한 상황

**Divide-and-Prompt를 사용하는 경우:**
- 높은 정확도가 필요한 상황
- 복잡한 비즈니스 로직
- 중요한 의사결정용 데이터 조회

### 다음 단계
- 실제 비즈니스 데이터로 테스트
- 프롬프트 템플릿 최적화
- 사용자 피드백을 통한 개선
- 오류 처리 로직 강화

**수고하셨습니다!**